In [ ]:
%load_ext cython

In [ ]:
%%cython -a
from libc.stdlib cimport rand, srand, RAND_MAX
cimport cython

cpdef monte_carlo(quantity, seed):
    return cdef_monte_carlo(quantity, seed)

@cython.cdivision(True)
cdef int cdef_monte_carlo(int quantity, int seed):
    cdef int hits = 0
    cdef double width = 0.0
    cdef double height = 0.0
    cdef double temp = 0.0
    srand(seed)
    
    for _ in range(quantity):
        width = <double>rand()/RAND_MAX
        height = <double>rand()/RAND_MAX
        if width * width + height * height < 1:
            hits += 1

    return hits

In [ ]:
def worker_monte_carlo(q_in, q_out):
    while True:
        arguments = q_in.get()
        x, y = arguments
        result = monte_carlo(x,y)
        q_out.put(result)
        q_in.task_done()

In [ ]:
import multiprocessing
import math
import random
nrOfCores = multiprocessing.cpu_count()
print('nrOfCores:', nrOfCores)

l = []
for i in range (128):
    random_number = int(random.random() * (2**31 -1))
    l.append((2_500_000, random_number))

In [ ]:
in_queue = multiprocessing.JoinableQueue()
result_queue = multiprocessing.Queue()

processes = []
for i in range(nrOfCores):
    p = multiprocessing.Process(target = worker_monte_carlo, 
                                args = (in_queue, result_queue))
    processes.append(p)
    p.start()
    
for parameter_set in l:
    in_queue.put(parameter_set)
    
import time

in_queue.join()

result_list = []
while not result_queue.empty():
    result_list.append(result_queue.get())

print(result_list)

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result_tuple in result_list:
    hits += result_tuple

    
print((hits)/trys * 4)
for p in processes:
    p.terminate()

## TimeIt

In [ ]:
%%timeit

import multiprocessing
in_queue = multiprocessing.JoinableQueue()
result_queue = multiprocessing.Queue()

processes = []
for i in range(nrOfCores):
    p = multiprocessing.Process(target = worker_monte_carlo, 
                                args = (in_queue, result_queue))
    processes.append(p)
    p.start()
    
for parameter_set in l:
    in_queue.put(parameter_set)
    
import time

in_queue.join()

result_list = []
while not result_queue.empty():
    result_list.append(result_queue.get())

print(result_list)

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result in result_list:
    hits += result
    
print((hits)/trys * 4)
for p in processes:
    p.terminate()
    
#5.96 s ± 33.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#6.03 s ± 70.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.9 s ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

#without hypot
#1.94 s ± 5.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)